In [1]:
#---------------------------------------------------------------------------
#                ___________
#    ______     /   ____    ____      _          ______
#   |  ____|   /   /    INRS    \    | |        |  ____|
#   | |       /   /     Edge     \   | |        | |
#   | |____  /   /    Computing   \  | |        | |____
#   |  ____| \   \  Communication /  | |        |  ____|   
#   | |       \   \   Learning   /   | |        | |
#   | |____    \   \_____LAB____/    | |_____   | |____
#   |______|    \ ___________        |_______|  |______|
#
#  Edge Computing, Communication and Learning Lab - INRS University
#
#  Author: Mobin Vaziri
#
#  Project: HENNC
#  
#  Creation Date: 2023-07-08
#
#  Description:  
#---------------------------------------------------------------------------

In [2]:
from sklearn.preprocessing import normalize
from matplotlib import pyplot as plt
from scipy.integrate import odeint
from tensorflow import keras
from keras.models import Sequential
from keras.models import load_model
from keras.utils import np_utils
from keras.layers.core import Dense, Activation
import numpy as np
import sklearn
import sklearn.metrics as metrics

In [3]:
a, b, c, d = 25, 33, 11, 6       # Constants
x_0, y_0, z_0 = 1.2, 0.6, 1.8    # Initial Conditions


def chaotic_system (X, t, a, b, c, d):
    
    x, y, z = X
    
    dxdt = a * (y - x) + (30 * y * z)
    dydt = (b * x) + (c * y) - (x * z)
    dzdt = (0.5 * x * z) - (d * z) + (x ** 2)
    
    return (dxdt, dydt, dzdt)

In [4]:
def solve_equation (img_h, img_w):
    
    Tot = (img_h * img_w)
    
    
    x = np.array ((img_h, img_w))
    y = np.array ((img_h, img_w))
    z = np.array ((img_h, img_w))
    
    tmax = 100
    t = np.linspace (0, tmax, Tot)
    f = odeint (chaotic_system, (x_0, y_0, z_0), t, args = (a, b, c, d))
    
    x, y, z = f.T
    
    x = x[:(Tot)]
    y = y[:(Tot)]
    z = z[:(Tot)]
    
    return (x, y, z)

In [5]:
def load_dataset (h: int, w: int):
    
    t_x, t_y, t_z = solve_equation (h + 1, w + 1)
    
    inp_x = t_x [0 : (h * w)]
    inp_y = t_y [0 : (h * w)]
    inp_z = t_z [0 : (h * w)]

    out_x = t_x [1 : ((h * w) + 1)]
    out_y = t_y [1 : ((h * w) + 1)]
    out_z = t_z [1 : ((h * w) + 1)]
    
    x = np.column_stack((inp_x,inp_y,inp_z))
    y = np.column_stack((out_x,out_y,out_z))
    
    return (x, y)

In [6]:
training_data, target_data = load_dataset (512, 512)

print(training_data.shape)
print(target_data.shape)

(262144, 3)
(262144, 3)


In [7]:
training_data_normalize = normalize (training_data)
target_data_normalize = normalize (target_data)

model = Sequential()
  
model.add(Dense(8, input_dim = 3))
model.add(Activation('relu'))

model.add(Dense(3))
model.add(Activation('linear'))

model.compile(loss = 'mean_absolute_error', optimizer = 'adam', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 32        
                                                                 
 activation (Activation)     (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 3)                 27        
                                                                 
 activation_1 (Activation)   (None, 3)                 0         
                                                                 
Total params: 59
Trainable params: 59
Non-trainable params: 0
_________________________________________________________________


In [8]:
#history = model.fit(training_data_normalize, target_data_normalize, epochs = 5, validation_split = 0.2, batch_size = 100)

In [9]:
trained_model = load_model("my_model.h5", compile=False)

In [10]:
output = trained_model.predict(training_data_normalize)

8192/8192 [==============================] - 10s 919us/step


In [11]:
mae = metrics.mean_absolute_error (target_data_normalize, output)
mse = metrics.mean_squared_error(target_data_normalize, output)
rmse = np.sqrt(mse)
r2 = metrics.r2_score(target_data_normalize, output)

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("R-SQUARED: ", r2)

MAE:  0.004101541125328822
MSE:  9.203218682840582e-05
RMSE:  0.009593340754315246
R-SQUARED:  0.9950916918858299


In [12]:
w_l1 = trained_model.layers[0].get_weights()[0]
b_l1 = trained_model.layers[0].get_weights()[1]

w_l2 = trained_model.layers[2].get_weights()[0]
b_l2 = trained_model.layers[2].get_weights()[1]

w_1 = list()
b_1 = list()

w_2 = list()
b_2 = list()

for r in range (0, w_l1.shape[0]):
    for c in range (0, w_l1.shape[1]):
        
        w_1.append(w_l1[r][c])

w_1 = np.array(w_1)

w_1 = w_1.reshape((w_l1.shape[0],w_l1.shape[1]))


for r in range (0, w_l2.shape[0]):
    for c in range (0, w_l2.shape[1]):
        
        w_2.append(w_l2[r][c])

w_2 = np.array(w_2)

w_2 = w_2.reshape((w_l2.shape[0],w_l2.shape[1]))


for i in range (0, b_l1.shape[0]):
    
    b_1.append(b_l1[i])
    
b_1 = np.array(b_1)

b_1 = b_1.reshape(b_l1.shape[0])


for i in range (0, b_l2.shape[0]):
    
    b_2.append(b_l2[i])
    
b_2 = np.array(b_2)

b_2 = b_2.reshape(b_l2.shape[0])

print("Shape of w_1 is {}, w_2 is {}, b_1 is {}, and b_2 is {}" .format(w_1.shape, w_2.shape, b_1.shape, b_2.shape))

np.savetxt('w_1.txt', w_1, delimiter=' ', fmt="%s")
np.savetxt('w_2.txt', w_2, delimiter=' ', fmt="%s")
np.savetxt('b_1.txt', b_1, delimiter=' ', fmt="%s")
np.savetxt('b_2.txt', b_2, delimiter=' ', fmt="%s")

Shape of w_1 is (3, 8), w_2 is (8, 3), b_1 is (8,), and b_2 is (3,)
